In [3]:
using JuMP, Gurobi, DataFrames, CSV
const GRB_ENV = Gurobi.Env()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-02-06


Gurobi.Env(Ptr{Nothing} @0x0000000135016c00, false, 0)

In [21]:
import Pkg; Pkg.add("Shapefile")
using Shapefile

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed DBFTables ──── v1.0.0
   Installed Qt5Base_jll ── v5.15.3+1
   Installed Shapefile ──── v0.7.4
   Installed GeoInterface ─ v0.5.7
    Updating `~/.julia/environments/v1.7/Project.toml`
  [8e980c4a] + Shapefile v0.7.4
    Updating `~/.julia/environments/v1.7/Manifest.toml`
  [75c7ada1] + DBFTables v1.0.0
  [cf35fbd7] + GeoInterface v0.5.7
  [8e980c4a] + Shapefile v0.7.4
  [ea2cea3b] ↑ Qt5Base_jll v5.15.3+0 ⇒ v5.15.3+1
Precompiling project...
  ✓ GeoInterface
  ✓ DBFTables
  ✓ Shapefile
  ✓ Qt5Base_jll
  ✓ GR_jll
  ✓ GR
  ✓ Plots
  7 dependencies successfully precompiled in 43 seconds (174 already precompiled)


In [19]:
m = Model(() -> Gurobi.Optimizer(GRB_ENV))

drc_data = CSV.read("data-CD-Oncho-iu.csv", DataFrame)
drc_data[!, :"ivermectin_amt"] = drc_data."PopReq" .* 9.27

drc_loc_T = Shapefile.Table("cod_admbnda_rgc_itos_20190911_shp/cod_admbndp_admALL_rgc_itos_20190911.shp")
drc_loc = DataFrame(drc_loc_T)

num_counties = length(drc_data."ADMIN2")
num_vehicles = 5
max_capacity = 10 #some viable constraint on the max number of mg that one vehicle can carry

drc_loc_x = drc_loc[!, "POINT_X"]
drc_loc_y = drc_loc[!, "POINT_Y"]
drc_dists = [sqrt((drc_loc_x[i] - drc_loc_x[j])^2 + (drc_loc_y[i] - drc_loc_y[j])^2) for i = 1:num_counties, j = 1:num_counties]

# variables
@variable(m, s_ij[i = 1:num_counties, j = 1:num_counties], Bin)   # path from i->j in service

@variable(m, v_in[i = 1:num_counties, n = 1:num_vehicles], Bin)   # vehicle n services location i

# constraints
@constraint(m, [n=1:max_capacity], sum(v_in[:, n]) <= max_capacity)   # vehicle doesn't explode


# objective

LoadError: InterruptException: